ECE 8803 Final Project

In [5]:
import numpy as np
import pandas as pd
import os
from dataloader import OCTDataset
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
import argparse
import os
import copy
import matplotlib.pyplot as plt



root = os.getcwd()
train = os.path.join(root, "df_prime_train.csv")
test = os.path.join(root, "df_prime_test.csv")



LABELS_SEVERITY = {35: 0,
                   43: 0,
                   47: 1,
                   53: 1,
                   61: 2,
                   65: 2,
                   71: 2,
                   85: 2}






'''Obtain Dataset Images'''


mean = (.1706)
std = (.2112)
normalize = transforms.Normalize(mean=mean, std=std)

transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    normalize,
])

args = 'C:/Users/jgril/Documents/GitHub/8803_Final_Project'
image_trainset = OCTDataset(args, 'train', transform=transform)
image_testset = OCTDataset(args, 'test', transform=transform)


print("loading data")
images = []
labels = []
for i in range(len(image_trainset)):
    image,label = image_trainset.__getitem__(i)
    images.append(image.flatten())
    labels.append(label)

assert(len(image_trainset) == len(images))
print("data loaded")



'''Meta Train Data'''
meta_trainset = pd.read_csv(train).dropna()
meta_testset = pd.read_csv(test).dropna()

train = os.path.join(root, "df_prime_train.csv")
test = os.path.join(root, "df_prime_test.csv")

meta_X_train = meta_trainset[['Age', 'Gender', 'Race', "Diabetes_Type", "Diabetes_Years", "BMI", "BCVA", "CST", "Leakage_Index"]].to_numpy()
meta_Y_train = meta_trainset[["DRSS"]].to_numpy()
y = []
for i in range(meta_Y_train.shape[0]):
    y.append(LABELS_SEVERITY[int(meta_Y_train[i])])

meta_Y_train = np.array(y)



'''Meta Test Data'''
meta_X_test = meta_testset[['Age', 'Gender', 'Race', "Diabetes_Type", "Diabetes_Years", "BMI", "BCVA", "CST", "Leakage_Index"]].to_numpy()
meta_Y_test = meta_testset[["DRSS"]].to_numpy() 
y = []
for i in range(meta_Y_test.shape[0]):
    y.append(LABELS_SEVERITY[int(meta_Y_test[i])])

Y_test = np.array(y)


#print(meta_X_train)
#print(meta_Y_train)
#print(meta_X_test)
#print(meta_Y_test)

Naive Bayes Implementation

In [2]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# load image data and corresponding labels
X_train = np.load('train_images.npy')
y_train = np.load('train_labels.npy')
X_test = np.load('test_images.npy')
y_test = np.load('test_labels.npy')

# convert image data to a bag-of-words representation
X_train_bow = np.zeros((X_train.shape[0], 256))
for i in range(X_train.shape[0]):
    unique, counts = np.unique(X_train[i], return_counts=True)
    X_train_bow[i, unique] = counts

X_test_bow = np.zeros((X_test.shape[0], 256))
for i in range(X_test.shape[0]):
    unique, counts = np.unique(X_test[i], return_counts=True)
    X_test_bow[i, unique] = counts

# train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_bow, y_train)

# evaluate the classifier on the test data
y_pred = clf.predict(X_test_bow)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

means
y=0 [ 53.1275692    0.48972321   0.51616881   0.87914497  17.21512743
  32.6912821   85.40367224 256.84091532   1.13702795]
y=1 [ 49.95196507   0.38864629   0.45851528   0.84279476  16.26637555
  30.93566433  84.60262009 259.12227074   1.35052402]
y=2 [ 44.09574468   0.4787234    0.46808511   0.92553191  16.81914894
  31.57008372  83.57446809 253.68085106   2.63021277]

     std
y=0 [ 9.50736566  0.49989438  0.86357679  0.32595873  8.37890912  4.20488318
  8.62727152 19.63223918  1.23580315]
y=1 [11.9647356   0.48744266  0.70201175  0.36399417  7.99529094  8.06877216
  6.9290977  24.51757915  1.46639227]
y=2 [10.22519744  0.4995471   0.51986348  0.26253112  5.63983319 12.12775823
 10.27337952 24.29876661  2.31756015]
Accuracy Score: 0.414


Polynomial Regression

SVM

KeyboardInterrupt: 